# Video Processing for BT Lab
Converts a video into png files. 
<br>
<br>
Usage: run notebook 
<br>
<br>
Notes: looks for a couple of JSON files to find video data.
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import cv2
import json
import pandas as pd
import numpy as np
from timeit import default_timer as timer
import psutil
import multiprocessing
import traceback

In [2]:
# make sure to update path
user_drive = input("Enter user drive: ").upper()
video_path = f"{user_drive}:/Christian/DI_centre_structured"
input(f"Is this the right directory - {video_path}?")

''

In [3]:
VIDEO_CHARACTERISTICS = {
    "With Blankets" : "WB",
    "B" : "WB",
    "Without Blankets" : "WOB",
    "WOB": "WOB",
    "3 Meters" : "3m",
    "2 Meters" : "2m",
    "Hold Breath" : "HB",
    "Hold Breathe" : "HB",
    "H" : "HB",
    "Relaxed" : "rel",
    "R": "rel",
}

In [4]:
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [5]:
def get_id(patient_data: dict, video_count:int) -> str:
    alias = patient_data["alias"]
    blanket = VIDEO_CHARACTERISTICS.get(patient_data["blanket"], "?")
    distance = VIDEO_CHARACTERISTICS.get(patient_data["distance"].title(), "?")
    breathing = VIDEO_CHARACTERISTICS.get(patient_data["breathing"], "?")
    id = alias + "-" + distance + "-" + blanket + "-" + breathing
    id = f"{alias}_{video_count}-{distance}-{blanket}-{breathing}"
    return id

In [6]:
""" creates folder for exported frames if it does not exist """
def set_folder(save_folder:str) -> None:
    if not os.path.isdir(save_folder):
        os.mkdir(save_folder)
    else:
        for files in os.listdir(save_folder):
            os.remove(os.path.join(save_folder, files))

In [7]:
class NotEnoughFreeSpace(Exception):
    pass

In [8]:
""" gets drives storage """
def get_drive_storage(user_drive:str):
    try:
        drive_usage = psutil.disk_usage(user_drive + ":\\")
        
        # Convert bytes to GB
        total_space = drive_usage.total / (1024**3)
        used_space = drive_usage.used / (1024**3)
        free_space = drive_usage.free / (1024**3)
        
        return {
            'drive_letter': user_drive,
            'total_space_GB': total_space,
            'used_space_GB': used_space,
            'free_space_GB': free_space,
        }
    except FileNotFoundError:
        return f"Drive {user_drive} not found."

def check_drive_usage(user_drive:str):
    minimum_space = 75
    drive_info = get_drive_storage(user_drive)

    if drive_info["free_space_GB"] < minimum_space:
        raise NotEnoughFreeSpace(f"Not enough free space on Drive {user_drive}. Minimum {minimum_space} GB required.")    

In [9]:
""" converts video into frames """
def run_video_to_frame(video_path:str , save_folder:str, frame_frequency: dict, new_fps:int, patient_id:str) -> list["str"]:
    # open video file
    video = cv2.VideoCapture(video_path)
    assert video.isOpened()

    # unpack metadata
    vid_fps = int(video.get(cv2.CAP_PROP_FPS))
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    vid_length_floored = number_of_frames // vid_fps

    # local variables
    pick_counter, save_counter, set_counter, true_frames = [0, 0, 0, 0]
    list_of_frames = [frame for frame, frequency in frame_frequency.items()]
    expected_frames = new_fps*vid_length_floored
    success = True

    # only for testing
    frame_limit = 100

    # make folder if it doesn't exist
    set_folder(save_folder)
    
    #  print to terminal
    print(f"\nSaving frames to: {save_folder}\nPicking frames: {list_of_frames} per set\nExpected number of frames: {expected_frames} ({new_fps}FPS * {vid_length_floored}s).")
    
    # start video processing
    start_time  = timer()
    while success and set_counter < vid_length_floored:
        success, frame = video.read()

        if not success:
            break

        if pick_counter in list_of_frames:
            frame_name = f"{patient_id}_{save_counter}.png"
            frame_path = os.path.join(save_folder, frame_name)
            num_of_save = frame_frequency.get(pick_counter)

            for _ in range(num_of_save):
                cv2.imwrite(frame_path, frame)
                save_counter += 1

        # update counters
        pick_counter += 1
        true_frames += 1
        if (pick_counter == vid_fps):
            set_counter += 1
            pick_counter = 0

        # for testing
        # if true_frames > frame_limit:
        #     print(f"\nLimit set to {frame_limit} frames for testing purposes.")
        #     break
    
    video.release()
    
    # get timing for one export
    end_time = timer()
    time_delta = end_time - start_time
    print(f"\nDone in {time_delta} seconds.")
    
    if save_counter != expected_frames:
        raise ValueError(f"Expected {expected_frames} frames, but got {save_counter} frames")

    return [set_counter, save_counter, true_frames]

In [10]:
""" gets folder paths """
def get_frames_path(local_path: str, level:str) -> list["str"]:
    fixed_path = local_path.replace("\\", "/")
    fixed_path_split = fixed_path.split("/")
    video_folder = "/".join(fixed_path_split[:-1])
    video_filename = fixed_path_split[-1].split(".")[0]
    folder_path = video_folder + f"/frames_{video_filename}_{level}"
    return [folder_path, video_folder]

In [11]:
""" re samples number of frames """
def up_sample(old_fps: int, new_fps: int, start:int) -> list[int]:
    frames_arr = np.arange(start, old_fps, dtype=int)
    frames_interp = np.linspace(start, old_fps-1, new_fps)
    nearest_indices = np.round(frames_interp).astype(int)
    up_sampled_list = np.take(frames_arr, nearest_indices, mode='wrap')

    return up_sampled_list.tolist()

In [12]:
def is_png_corrupted(file_path):
    try:
        image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        if image is None:
            return True
        return False
    except Exception:
        return True

def check_for_corrupted_png_files(folder_path):
    corrupted_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".png"):
                file_path = os.path.join(root, file)
                if is_png_corrupted(file_path):
                    corrupted_files.append(file_path)
    
    if len(corrupted_files) != 0:
        print("Corrupted PNG files found:")
        for file_path in corrupted_files:
            print(file_path)           
    
    return corrupted_files

In [13]:
""" driver code for converting a video into frames """
def convert_video_to_frame(all_patients:dict, level: str, new_fps:int, user_drive:str) -> None:
    visited_folders = {}    

    for json_index, patient_info in all_patients.items():
        try:
            video_path = patient_info["local path"]
            old_fps = int(patient_info["old fps"])

            frames_folder, video_folder = get_frames_path(video_path, level)
            visited_folders[video_folder] = visited_folders.get(video_folder, 0) + 1
            video_count = visited_folders[video_folder]

            frames_to_pick = up_sample(old_fps, new_fps, 1)
            frames_idx = pd.Index(frames_to_pick, name="frames")
            frame_frequency = frames_idx.value_counts()

            if len(frames_to_pick) != new_fps:
                raise ValueError("Number of frames to pick is not equal to new fps")

            # get id
            patient_id = get_id(patient_info, video_count)

            # run video to frames
            set_counter, save_counter, true_frames = run_video_to_frame(video_path, frames_folder, frame_frequency, new_fps, patient_id)

            #display output
            print(f"\nSet counter: {set_counter}, save counter: {save_counter}, frame counter: {true_frames}\n\n"+ "-"*50)

            # check if folder has any corrupted files
            corrupted_files = check_for_corrupted_png_files(frames_folder)

            # exit if drive storage has less than 50 GB
            check_drive_usage(user_drive)

        except Exception as e:
            traceback.print_exc()
            print(f'''{type(e)}: {e} for video {patient_info["filename"]}''')

        break

In [14]:
""" reads JSON file """
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

## Video to frames (stage 3.1)

Based on the desired fps, turn videos into frames

In [15]:
""" local vals"""

rgb_fps = {
    "lower_bound": 10,
    "upper_bound": 20
}

thermal_fps = {
    "lower_bound": 5,
    "upper_bound": 10
}

In [16]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "/testing/rgb_test.json")
metadata_thermal = load_json(json_dir, "/testing/thermal_test.json")

In [17]:
""" convert video to png (rgb) """

for level, new_fps in rgb_fps.items():
    print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
    convert_video_to_frame(metadata_rgb, level, new_fps, user_drive)
    break


Adjusting FPS to 10

Saving frames to: C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/frames_Arun2_lower_bound
Picking frames: [2, 4, 6, 7, 9, 11, 13, 14, 16, 1] per set
Expected number of frames: 70 (10FPS * 7s).

Done in 9.291371099999992 seconds.

Set counter: 7, save counter: 70, frame counter: 126

--------------------------------------------------
